# Introduction à l'apprentissage automatique: TP3 - Exercice 1 - <font color=red> CORRECTION </font>

<br>

## Détection de spam

<br>

Dans ce TP, nous allons entraîner des classifieurs pour décider si un mail est un spam ou non.

<br>

Tout d'abord, quelques indications sur l'utilisation des méthodes d'apprentissage de `scikit-learn`.

Les méthodes d'apprentissage supervisé de `scikit-learn` permettent de définir un objet, doté de différents attributs et méthodes, dont `cross_val_score` (pour calculer un score de validation croisée), `fit` (pour procéder à l'apprentissage), `predict` (pour prédire les classes des éléments d'une base de test), ou `score` pour calculer la proportion d'observations bien classées dans la base de test, sur laquelle on peut comparer la classe prédite à la "vraie" classe.

Ci-dessous, un exemple d'utilisation de la classification au plus proche voisin, dans un scénario où on suppose disposer d'une base d'apprentissage $(X_{train},y_{train})$, et d'une base de test $X_{test}$ pour laquelle on connaît $y_{test}$, de manière à valider l'apprentissage sur la base de test. Si on veut changer de classifieur, il suffit d'utiliser un autre constructeur que `neighbors.KNeighborsClassifier` et de passer les paramètres adéquats.

```python
# (le code suivant ne peut pas être exécuté "tel quel"...)

# classifieur au plus proche voisin (on peut changer le paramètre n_neighbors):
knn = neighbors.KNeighborsClassifier(n_neighbors=1)  

# calcul d'un score moyen de validation croisée "à 5 plis" sur (X_train,y_train)
scores = cross_val_score(knn,X_train,y_train,cv=5)
print("score moyen de validation croisée: %0.3f (+/- %0.3f)" % (scores.mean(),2*scores.std()))

# la prédiction d'une nouvelle observation consistera à chercher le p.p.v. dans X_train, 
# et à associer la classe de ce p.p.v., donnée par y_train:
knn.fit(X_train,y_train)  
# Remarque: il n'y a pas d'apprentissage à proprement parler pour les p.p.v., 
# il s'agit juste de préciser la base dans laquelle seront cherchés les plus proches voisins

# on stocke dans y_pred les classes prédites sur un ensemble de test X_test:
y_pred = knn.predict(X_test)  

# calcul d'un score lorsqu'on connaît les vraies classes des observations de X_test: 
# (proportion d'observations pour lesquelles y_test==y_pred)
score = knn.score(X_test,y_test)
```

## 1. Préliminaires

<br>

Commençons par charger les bibliothèques utiles au TD.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import neighbors, linear_model, naive_bayes, metrics

%matplotlib inline

Ensuite, on charge les données: récupérez au préalable le fichier `spambase.data` disponible sur le [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Spambase) (cliquez sur "Download"). La description complète de la base est dans le fichier `spambase.name`, à ouvrir avec un éditeur de texte.

<br>

La cellule suivante charge les données. On forme une base d'entraînement avec 80% des données (choix aléatoire), et on garde 20% des données pour faire une base de test. Dans la cellule suivante, on fixe la graîne du générateur aléatoire (`random_state=1`, la valeur est arbitraire) de manière à ce que l'on ait tous les mêmes résultats afin de faciliter la comparaison.


<font color=red>
    
_Remarque_ : les générateurs pseudo-aléatoires sont généralement des générateurs congruentiels linéaires. Voir: https://fr.wikipedia.org/wiki/G%C3%A9n%C3%A9rateur_de_nombres_pseudo-al%C3%A9atoires

Ils fournissent une séquence déterministe à partir d'un état initial donné (graine du générateur). Généralement, on fixe l'état initial `random_state` à partir du temps CPU, de manière à générer différentes séquences à chaque utilisation. De cette manière, chaque exécution (sur votre machine ou la machine d'un camarade) fournirait un résultat différent.

Ici, on fixe tous `random_state` à la même valeur, de manière à tous avoir la même répartition train/test, et pouvoir comparer nos résultats. C'est une astuce très utilisée en phase de déboguage. Attention néanmoins à ne pas en abuser, il y a des situations où il est crucial de ne pas garder la même initialisation à chaque exécution. Par exemple, dans les algorithmes d'optimisation stochastiques (par exemple le gradient stochastique que l'on verra dans le cours sur les réseaux de neurones), il ne faut pas fixer l'état initial a priori. Dans ce dernier cas, ce qui nous intéresserait est le comportement en moyenne de l'optimisation, pas le comportement pour un choix donné d'état initial.
    
Une remarque cependant: attention, [`random_state` dépend de l'OS](https://github.com/lmcinnes/umap/issues/153), vous pouvez avoir des résultats différents de vos camarades sous Linux/MacOS/Windows avec la même valeur d'état initial... 

</font>

In [ ]:
data = np.loadtxt('spambase.data', delimiter=',')
X_train, X_test, y_train, y_test = train_test_split(data[:,:-1], data[:,-1], test_size=0.2, random_state=1)
# pour vérifier que les données sont bien chargées:
print("dataset:")
print(data)  
print("\nTOTAL - nombre d'observations, nombre de caractéristiques:")
print(data.shape)
print("\nAPPRENTISSAGE - nombre d'observations, nombre de caractéristiques:")
print(X_train.shape)
print("\nAPPRENTISSAGE - nombre de labels associés aux obervations:")
print(y_train.shape)
print("\nTEST - nombre d'observations, nombre de caractéristiques:")
print(X_test.shape)
print("\nTEST - nombre de labels associés aux obervations:")
print(y_test.shape)
print("\nobservations, base d'apprentissage:")
print(X_train)
print("\nlabels associés, base d'apprentissage:")
print(y_train)
print("\nproportion de spams dans la base d'apprentissage:")
print(np.mean(y_train))

__Question 1__. A partir de la description de la base de données, justifiez la manière employée pour charger les données en `X` (observations) et `y` (labels). Quelles sont les caractéristiques des observations, les labels, et quel est le rapport avec le problème initial?

<font color=red>
  
Les caractéristiques sont séparées par le délimiteur "," (ouvrir le fichier avec un éditeur de texte pour le vérifier).
    
D'après la description du fichier, chaque ligne est une observation (la description d'un mail) et regroupe 58 caractéristique, la dernière étant 1 si le message est un spam, 0 sinon.

Les 57 premières caractéristiques forment donc un descripteur `x`, et `y` est bien l'étiquette dans ce problème d'apprentissage supervisé.
     
La description des caractéristiques est donnée par `spambase.names` (voir aussi `spambase.DOCUMENTATION`) dans l'archive sur la page web de l'UCI. La méthode pour décider des mots dont on calculera les statistiques n'est pas précisée. On pourrait s'inspirer des méthodes de quantification par clustering et descripteurs TF-IDF vues au TP2 pour sélectionner le "sac de mots" sur lequel on calculerait des statistiques.
    
Chaque mail est donc décrit par un vecteur de ${\mathbb R}^{57}$, il s'agit d'un problème de classification biclasse (classe 0: non-spam; classe 1: spam).
    
</font>

__Remarque importante__: lorsqu'on teste des classifieurs, il est important de comparer les scores de classification obtenus à celui d'un "dummy classifier" (un classifieur fictif): un classifieur qui fait une prévision sans tenir compte des observations. Par exemple, ici un classifieur qui classerait toute observation comme "non spam" aurait raison dans presque 60% des cas. On espère donc que les classifieurs réels soient meilleurs. 

## 2. Classification aux plus proches voisins

<br>

Mettez en oeuvre les classifications au plus proche voisin et aux 5 plus proches voisins. Vous calculerez le score moyen de validation croisée à 5 plis sur la base d'apprentissage ainsi que le score obtenu sur la base de test. Vous vous inspirerez du code détaillé en introduction. 

__Question 2__. Quelle est la métrique utilisée pour déterminer les plus proches voisins? Quel est ce "score" calculé exactement? Quel lien entre score de validation croisée et score sur la base de test?

In [ ]:
print("Classification 1-ppv:")
knn = neighbors.KNeighborsClassifier(n_neighbors=1)  
scores_knn = cross_val_score(knn,X_train,y_train,cv=5)  
print("score moyen de validation croisée: %0.3f (+/- %0.3f)" % (scores_knn.mean(),2*scores_knn.std()))
knn.fit(X_train,y_train)  # (il n'y a pas d'apprentissage à proprement parler pour les p.p.v.)
y_pred_knn = knn.predict(X_test)  
score_knn = knn.score(X_test,y_test)
print("score sur la base de test: %0.3f" %score_knn)

In [ ]:
print("Classification 5-ppv:")
knn5 = neighbors.KNeighborsClassifier(n_neighbors=5)  
scores_knn5 = cross_val_score(knn5,X_train,y_train,cv=5)
print("score moyen de validation croisée: %0.3f (+/- %0.3f)" % (scores_knn5.mean(),2*scores_knn5.std()))
knn5.fit(X_train,y_train) 
y_pred_knn5 = knn5.predict(X_test)  
score_knn5 = knn5.score(X_test,y_test)
print("score sur la base de test: %0.3f" %score_knn5)

<font color=red>


D'après 
    https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
la distance euclidienne (entre vecteurs de ${\mathbb R}^{57}$ ici) est utilisée par défaut pour déterminer les plus proches voisins.

D'après la documentation:
    https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
le score est donné par le "default scorer" du classifieur, qui est d'après:
    https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
la "mean accuracy" (précision moyenne), c'est à dire ici la proportion d'observations correctement classées.

Score de validation croisée et score sur la base de test ont des valeurs proches, ce qui est normal si la base de test est assez grande et bien représentative de la base d'apprentissage. Voir cours.

Attention, le score de validation croisée (cf cours séance 2) est obtenu comme moyenne des scores des classifieurs pour lesquels l'apprentissage est fait, tour à tour, sur 4/5 de la base d'apprentissage, le score étant calculé sur le dernier 1/5. Par contre, le score sur la base de test est celui d'un classifieur entraîné sur l'intégralité de la base d'apprentissage.

_Remarque_ : si on fait `print(scores_knn)` on affiche les scores de classifcation sur les 5 plis, qui servent ensuite à calculer le score moyen et son écart-type.

</font>

__Question 3__. Pourquoi la métrique utilisée n'est-elle pas adaptée aux observations ?

<font color=red>
    
La norme euclidienne donne le même poids à chaque caractéristique. Or, les caractéristiques ne varient pas dans le même intervalle: les 54 premières caractéristiques sont des pourcentages (entre 0 et 100), ensuite les caractéristiques sont des longueurs de chaînes de caractères. Une variation de 1% dans une caractéristique n'aura donc pas le même impact sur la norme selon la nature de la caractéristique.
    
Par ailleurs, de manière générale on a intérêt à travailler avec des caractéristiques dans une même gamme de valeur pour des questions de stabilité numérique des algorithmes d'estimation.

</font>

__Question 4__. Pré-traitez les données par standardisation, comme expliqué ici sur [la documentation scikit-learn](https://scikit-learn.org/stable/modules/preprocessing.html) (utilisez `StandardScaler` de manière à appliquer la même normalisation sur la base d'apprentissage et sur la base de test, c'est important), puis recalculez les scores des deux classifieurs précédents.

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_standard = scaler.transform(X_train)
X_test_standard = scaler.transform(X_test)

print("Classification 1-ppv après normalisation:")
knn = neighbors.KNeighborsClassifier(n_neighbors=1)  
scores_knn = cross_val_score(knn,X_train_standard,y_train,cv=5)  
print("score moyen de validation croisée: %0.3f (+/- %0.3f)" % (scores_knn.mean(),2*scores_knn.std()))
knn.fit(X_train_standard,y_train) 
y_pred_knn = knn.predict(X_test_standard)  
score_knn = knn.score(X_test_standard,y_test)
print("score sur la base de test: %0.3f" %score_knn)

print("\nClassification 5-ppv après normalisation:")
knn5 = neighbors.KNeighborsClassifier(n_neighbors=5)  
scores_knn5 = cross_val_score(knn5,X_train_standard,y_train,cv=5)
print("score moyen de validation croisée: %0.3f (+/- %0.3f)" % (scores_knn5.mean(),2*scores_knn5.std()))
knn5.fit(X_train_standard,y_train) 
y_pred_knn5 = knn5.predict(X_test_standard)  
score_knn5 = knn5.score(X_test_standard,y_test)
print("score sur la base de test: %0.3f" %score_knn5)


<font color=red>
    
On voit que la normalisation améliore grandement le score de classification. On ne peut pas distinguer les 1-ppv et 5-ppv, qui ont un score similaire.    

__Rappel__: la base test ne doit servir qu'à vérifier la performance des classifieurs, et elle ne doit pas intervenir dans l'entraînement du classifieur. On calcule donc moyenne et écart-type uniquement sur la base d'apprentissage (et pas sur la base d'apprentissage ET la base test), puis on applique la même normalisation sur les bases d'apprentissage et de test: l'objectif est de pouvoir mesurer la performance sur la base test du classifieur entraîné sur la base d'apprentissage normalisée.
    
__Attention à ne pas normaliser les `y`__: ils codent la classe en 0 ou 1, cela n'a pas de sens de chercher à les normaliser.
    
</font>

## 3. Classifieur naïf de Bayes gaussien et classifieur de la régression logistique

<br>

__Question 5__. On __admettra__ que le classifieur naïf de Bayes gaussien ne nécessite pas de standardisation préalable des données. La démonstration figurera dans la correction. (vous pouvez vérifier en essayant avec ou sans normalisation que la normalisation joue tout de même un faible rôle: elle a sans doute une influence sur le comportement de l'algorithme d'estimation des paramètres)

Mettez en oeuvre le classifieur naïf de Bayes gaussien (lisez le début de la [documentation](https://scikit-learn.org/stable/modules/naive_bayes.html) où vous retrouverez le contenu du cours, puis la syntaxe de `GaussianNB` [ici](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)).

<font color=red>
    
__Justification de l'inutilité de standardiser les données avec GNB (complément d'information)__

Dans le classifieur GNB, on modélise les distributions conditionnelles des composantes comme des gaussiennes:
$$ p(x_i| C_k) = \frac{1}{\sqrt{2\pi}\sigma_{ik}} e^{-|x_i-\mu_{ik}|^2/(2\sigma_{ik}^2)}$$

Ensuite, on classe une observation $(x_1,x_2,\dots,x_{57})$ dans la classe 0 si:
$$ p(C_0) \prod_{i=1}^{57} p(x_i| C_0) > p(C_1) \prod_{i=1}^{57} p(x_i| C_1)$$
soit:
$$ p(C_0) \frac{1}{(2\pi)^{57/2}\sigma_{0}} e^{-\frac12 \sum_{i=1}^{57} ((x_i-\mu_{i0})/\sigma_{i0})^2} > 
p(C_1) \frac{1}{(2\pi)^{57/2}\sigma_{1}} e^{-\frac12 \sum_{i=1}^{57} ((x_i-\mu_{i1})/\sigma_{i1})^2}$$
où $\sigma_k = \prod_{i=1}^{57} \sigma_{ik}$
    
En passant au logarithme:
$$ \log(p(C_0)) - \log(\sigma_0) -\frac12 \sum_{i=1}^{57} \frac{(x_i-\mu_{i0})^2}{\sigma_{i0}^2} > \log(p(C_1)) - \log(\sigma_1) -\frac12 \sum_{i=1}^{57} \frac{(x_i-\mu_{i1})^2}{\sigma_{i1}^2} \qquad \text{(règle R1)}$$

    
Supposons à présent qu'on travaille sur les données normalisées. Chaque caractéristique est transformée selon $\tilde{x}_i = (x_i - m_i)/s_i$ où $m_i$ et $s_i$ sont la moyenne et l'écart-type empiriques de la $i$-ème caractéristique.    

Sur les données normalisées, la règle de décision fournie par GNB serait de classer une observation normalisée $(\tilde{x}_1,\tilde{x}_2,\dots,\tilde{x}_{57})$ dans la classe 0 si:
$$\log(p(C_0)) - \log(\tilde{\sigma}_0) -\frac12 \sum_{i=1}^{57} \frac{(\tilde{x}_i-\tilde{\mu}_{i0})^2}{\tilde{\sigma}_{i0}^2} > \log(p(C_1)) - \log(\tilde{\sigma}_1) -\frac12 \sum_{i=1}^{57} \frac{(\tilde{x}_i-\tilde{\mu}_{i1})^2}{\tilde{\sigma}_{i1}^2} \qquad \text{(règle R2)}$$
c'est-à-dire (en remplaçant les $\tilde{x}_i$ par $(x_i - m_i)/s_i$), si:
$$\log(p(C_0)) - \log(\tilde{\sigma}_0) -\frac12 \sum_{i=1}^{57} \frac{(x_i-m_i-\tilde{\mu}_{i0}s_i)^2}{\tilde{\sigma}_{i0}^2s_i^2} > \log(p(C_1)) - \log(\tilde{\sigma}_1) -\frac12 \sum_{i=1}^{57} \frac{(x_i-m_i-\tilde{\mu}_{i1}s_i)^2}{\tilde{\sigma}_{i1}^2s_i^2}$$
    
Maintenant, $\tilde{\mu}_{i0}$ est la moyenne empirique de la i-ème caractéristique des observations normalisées appartenant à la classe 0. Ainsi, $\tilde{\mu}_{i0} = (\mu_{i0}-m_i)/s_i$ (pareil pour la classe 1).
    
De même,  $\tilde{\sigma}_{i0}^2$ est la variance empirique de la i-ème caractéristique des observations normalisées appartenant à la classe 0. Ainsi, $\tilde{\sigma}_{i0}^2 = \sigma_{i0}^2/s_i^2$ (pareil pour la classe 1).
    
L'inégalité précédente s'écrit donc:
$$\log(p(C_0)) - \log(\tilde{\sigma}_0) -\frac12 \sum_{i=1}^{57} \frac{(x_i-\mu_{i0})^2}{\sigma_{i0}^2} > \log(p(C_1)) - \log(\tilde{\sigma}_1) -\frac12 \sum_{i=1}^{57} \frac{(x_i-\mu_{i1})^2}{\sigma_{i1}^2}$$
   
Enfin, $\log(\tilde{\sigma}_0) =  \log\left( \prod_{i=1}^{57} \tilde{\sigma}_{i0}\right) =  \log(\sigma_0) - \log\left( \prod_{i=1}^{57} s_i^2 \right)$

Donc:
$$\log(p(C_0)) - \log(\sigma_0) -\frac12 \sum_{i=1}^{57} \frac{(x_i-\mu_{i0})^2}{\sigma_{i0}^2} > \log(p(C_1)) - \log(\sigma_1) -\frac12 \sum_{i=1}^{57} \frac{(x_i-\mu_{i1})^2}{\sigma_{i1}^2}$$
On retrouve la règle R1.    
    
On peut conclure: la règle de décision R2 (GNB sur les données normalisées) et R1 (GNB sur les données originales) sont identiques.
    
Néanmoins, la procédure d'estimation des moyennes et écarts-types sont légèrement différentes entre `StandardScaler` et `GaussianNB` (estimateurs empiriques pour le premier, estimation du maximum de vraisemblance pour le second). Cela peut expliquer une légère différence en pratique. Le seul intérêt de la standardisation préalable pour GNB est d'éviter parfois des problèmes d'optimisation numérique.
    
</font>

In [ ]:
print("classification par GNB")
GNB = naive_bayes.GaussianNB()  
scores_GNB = cross_val_score(GNB,X_train,y_train,cv=5)
print("score moyen de validation croisée: %0.3f (+/- %0.3f)" % (scores_GNB.mean(),2*scores_GNB.std()))
GNB.fit(X_train,y_train)  
y_pred_GNB = GNB.predict(X_test)  
score_GNB = GNB.score(X_test,y_test)
print("score sur la base de test: %0.3f" %score_GNB)


# avec normalisation  (décommentez les lignes suivantes)
#print("\nclassification par GNB après normalisation")
#GNB = naive_bayes.GaussianNB()  
#scores_GNB = cross_val_score(GNB,X_train_standard,y_train,cv=5)
#print("score moyen de validation croisée: %0.3f (+/- %0.3f)" % (scores_GNB.mean(),2*scores_GNB.std()))
#GNB.fit(X_train_standard,y_train)  
#y_pred_GNB = GNB.predict(X_test_standard)  
#score_GNB = GNB.score(X_test_standard,y_test)
#print("score test: %0.3f" %score_GNB)

Mettez en oeuvre le classifieur de la régression logistique ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)). 

Passez l'option `max_iter=2000` si vous avez un avertissement concernant la convergence de l'optimisation, de la manière suivante:
`LR = linear_model.LogisticRegression(max_iter=2000)`

In [ ]:
# classifieur régression logistique (sur les données standardisées)

# d'après la documentation: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# il y a régularisation par défaut
# on fait une gridsearch (voir la documentation) pour trouver la valeur optimale de C par validation croisée 
# Ce n'est pas demandé, on peut se contenter de la valeur de C par défaut (C=1) ou penalty="none"

import matplotlib.pyplot as plt
print("Régression logistique: gridsearch\n")
LR = linear_model.LogisticRegression(max_iter=2000)   # il faut augmenter max_iter sinon on a des problèmes de convergence
GS=GridSearchCV(LR,{'C':[0.001,0.01,0.1,1,10,100, 1000, 10000, 100000, 1000000]},cv=5)
GS.fit(X_train_standard,y_train)
print("\nmeilleur estimateur:")
print(GS.best_estimator_)
print("\nscore de validation croisée pour chaque valeur de C:")
print(GS.cv_results_['mean_test_score'])
plt.figure()
plt.semilogx([0.001,0.01,0.1,1,10,100, 1000, 10000, 100000, 1000000],GS.cv_results_['mean_test_score'],'-*')
plt.title('score de validation croisée contre C')
plt.grid()
plt.show();

# on demandait uniquement le code suivant:
print("\nClassification par régression logistique")
LR = linear_model.LogisticRegression() #(max_iter=2000) à ajouter dans certaines versions de sklearn
scores_LR = cross_val_score(LR,X_train_standard,y_train,cv=5)
print("score moyen de validation croisée: %0.3f (+/- %0.3f)" % (scores_LR.mean(),2*scores_LR.std()))
%time LR.fit(X_train_standard, y_train)
score_LR = LR.score(X_test_standard, y_test)
print('score: %2f' %score_LR)
y_pred_lr = LR.predict(X_test_standard)

<font color=red>

_Remarque sur la validation croisée (gridsearch)_ : à partir de $C=0.1$, le mean_test_score ne change qu'au troisième chiffre après la virgule. On aurait tout aussi bien pu prendre $C=0.1$ (ou $C=1$, valeur par défaut suggérée par l'énoncé), et d'ailleurs on aurait pu avoir un optimum différent avec un autre nombre de pli ou tirage aléatoire des plis.
    
C'est une remarque valable dans toutes ces questions où on cherche le "meilleur" hyperparamètre: il ne faut pas chercher à départager au delà des fluctuations d'échantillonnage.
    
Encore une fois, le choix de l'hyperparamètre $C$ par validation croisée n'était pas demandé.
    
</font>

## 4. Analyse des résultats

<br>

On dispose des matrices de confusion, décrites [ici](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html), et des rapports de classification, décrits [là](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html).

__Question 6__. Affichez ces matrices et rapports sur la base test pour les quatre classifieurs étudiés.

In [ ]:
print("\nPlus proche voisin, données standardisées:")
print("score sur la base de test: %0.3f" %score_knn)
print(metrics.classification_report(y_test,y_pred_knn))
print("matrice de confusion:")
print(metrics.confusion_matrix(y_test,y_pred_knn))

print("\n5 plus proches voisins, données standardisées:")
print("score sur la base de test: %0.3f" %score_knn5)
print(metrics.classification_report(y_test,y_pred_knn5))
print("matrice de confusion:")
print(metrics.confusion_matrix(y_test,y_pred_knn5))

print("\nClassifieur naïf Gaussien:")
print("score sur la base de test: %0.3f" %score_GNB)
print(metrics.classification_report(y_test,y_pred_GNB))
print("matrice de confusion:")
print(metrics.confusion_matrix(y_test,y_pred_GNB))

print("\nRégression logistique:")
print('score sur la base de test: %0.3f' %score_LR)
print(metrics.classification_report(y_test,y_pred_lr))
print("matrice de confusion:")
print(metrics.confusion_matrix(y_test,y_pred_lr))

__Question 7__. Ici, par quoi pourraient s'expliquer les performances modestes du classifieur naïf de Bayes ? A ce stade, quel classifieur préfére-t-on et pourquoi? Dans une application de détection de spams, cherche-t-on réellement à minimiser le taux d'erreur global?

<font color=red>
  
Le classifieur naïf de Bayes suppose l'indépendance conditionnelle des caractéristiques. Cela ne permet pas de modéliser les corrélations entre caractéristiques. Or un spam est caractérisé par la fréquence élevée de plusieurs mots en même temps (co-occurrence). Par exemple, un mail avec plusieurs occurrences simultanées de "money", "receive", "viagra" est suspicieux; il le sera moins s'il contient l'un de ces mots mais pas les autres. Le classifieur naïf de Bayes se base uniquement sur la distribution des fréquences des mots considérés individuellement, pas sur leurs corrélations, ce qui explique les performances modestes.
    
<br>
    
__Attention__ à l'ordre des arguments dans les deux fonctions: `y_true`, puis `y_pred` (sinon on récupère par exemple la transposée de la matrice de confusion)
    
Cf [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)
    
* precision = tp / (tp+fp)
* recall = tp / (tp+fn)   (rappel)
* f1_score: 2 * precision * recall / (precision + recall)  (moyenne harmonique de precision et recall)
* support: nombre d'observations classées dans classe k

où tp = true positive (classé en $k$ à raison), fp = false positive (classé en $k$ à tort), fn = false negative (classé dans une autre classe à tort).
 
interprétation:  
* tp+fp: nombre d'observations classées dans la classe k (à raison + à tort)
* tp+fn: nombre d'observations réellement dans la classe k  (bien détectées + mal détectées)
* precision = proportion d'observations classées dans k à raison
* recall = proportion d'observations classées dans k parmi tous les éléments réellement dans la classe k  
 
Idéalement, on voudrait precision et recall proches de 1 (s'ils sont égaux à 1, fp=fn=0).
  
<br>

En ce qui concerne `confusion_matrix`, d'après la documentation, les vraies classes sont en lignes, les classes prédites en colonnes.
    
<br>
    
On voit par exemple que GNB classe 21 mails en non-spam alors qu'ils sont des spams, parmi les 21+336=357 spams. Par contre 148 mails ont été classés en spam alors qu'ils n'en sont pas, parmi les 148+416=564 non-spams.
    
<br>
    
Dans cette application, on souhaite ne pas classer à tort des messages "normaux" en spam (car ils n'arriveront alors jamais à l'utilisateur). Le critère de choix serait donc le rappel de la classe 0, que l'on veut le plus grand possible, plutôt que le score de classification.
    
Le classifieur de la régression logistique est donc la "meilleure" méthode parmi celles testées selon ce critère: 96% des non-spams sont bien classés. C'est également celle qui a le meilleur score global: dans 92% des cas, sa prédiction est correcte.
    
Au passage, on voit que GNB a la meilleure précision pour la classe 0: cela signifie qu'il est le meilleur pour détecter les non-spams (lorsqu'il détecte un non-spam, il se trompe peu). Le problème est que lorsqu'il prétend détecter un spam, il se trompe souvent. GNB a également le meilleur rappel pour la classe 1: un vrai spam est bien classé en spam. Ce n'est pas contradictoire avec la phrase précédente: le problème est que les détections en spam sont souvent en fait des non-spams.
    
</font>

## 5. Toutes les erreurs ne se valent pas...

Le classifieur bayésien naïf gaussien et le classifieur de la régression logistique s'appuient tous deux sur la règle du maximum a posteriori. Ils permettent d'estimer la probabilité a posteriori $p(C_1|x)$ et détectent un spam lorsque $p(C_1|x)>1/2$, où $C_1$ désigne la classe "spam" et $x$ est une observation. Les deux classifieurs mettent en oeuvre le classifieur de Bayes, qui minimise le risque moyen de prédiction (le taux d'erreur). Le taux d'erreur "compte" de la même manière les erreurs sur les deux classes.

Si on préfère réduire le taux de faux positif de la méthode (proportion de mails détectés à tort comme "spam"), on peut relever le seuil de cette probabilité.

Les classifieurs `LogisticRegression` et `GaussianNB` possèdent tous deux une méthode `predict_proba` qui, pour un tableau d'observations, fournit la probabilité a posteriori de chaque classe, comme l'affiche la cellule suivante. On remarque que pour chaque observation $x$, $p(C_0|x)+p(C_1|x)=1$.  (attention, la documentation n'est pas très claire, `predict_proba` fournit bien la probabilité a posteriori, et pas la vraisemblance $p(x|C_k)$)

Remarquons qu'aucune probabilité n'est fournie par la classification aux plus proches voisins.

In [ ]:
print("probabilités a posteriori pour GNB:")
print(GNB.predict_proba(X_test))
print("\nprobabilités a posteriori pour LR:")
print(LR.predict_proba(X_test_standard))

In [ ]:
# faites varier le seuil de détection p:
p=0.63  # constatez que p=0.5 fournit les mêmes résultats pour y_pred_lr et y_pred_LRb
y_pred_LRb = (LR.predict_proba(X_test_standard)[:,1] >= p).astype(int)
#print(y_pred_LRb)   # pour visualiser les classes prédites
#print(y_pred_lr)
score_LRb = 1-np.mean(np.abs(y_test-y_pred_LRb))  # calcul du taux de reconnaissance

print("\nClassification de la régression logistique pour un seuil de probabilité p=%.3f" %p)
print('score sur la base de test: %.3f' %score_LRb)
print(metrics.classification_report(y_test,y_pred_LRb))
print("matrice de confusion:")
print(metrics.confusion_matrix(y_test,y_pred_LRb))



__Question 8__. Quelle valeur du seuil de probabilité $p$ faut-il choisir pour assurer un rappel de la classe "non spam" d'au moins 0.98?
Que penser de cet algorithme de détection de spam?

<font color=red>
    
Il faut choisir $p=0.63$ pour LR dans la cellule précédente: 14 mails classés en spam à tort parmi 550+14 = 564 mails qui ne sont pas du spam.

On constate que le taux de reconnaissance (score) décroît par rapport à la classification obtenue avec le seuil $p=0.5$.

On a alors un rappel de la classe "spam" de 0.83: 62 mails ont été classés en non-spam alors qu'ils le sont parmi les 62+295 spams reçus.

<br>
    
__Conclusion__: lorsqu'on règle le seuil $p$ de l'algorithme de manière à mal classer moins de 2% des mails qui ne sont pas des spams, on détecte correctement un peu plus des trois-quarts des spams reçus. Ce sont des performances modestes. Néanmoins on est tributaire du "sac de mots" choisi initialement pour former le fichier sur lequel on travaille, que l'on ne maîtrise pas dans cet exercice.

Remarquons qu'il est très simple d'assurer un rappel de la classe "non spam" (on parle de sensibilité, _sensitivity_ ) de 1: il suffit de classer tout mail comme non-spam... Bien entendu, le rappel de la classe "spam" (on parle de spécificité, _specificity_ ) est alors nul. Toute méthode de détection doit être analysée simultanément selon ces deux critères, le seuil de détection devant être choisi selon un compromis entre sensibilité et spécificité.

__Pour aller plus loin__, lire ce qui concerne les __[courbes ROC](https://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics)__.

</font>
